In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

In [2]:
inputs = torch.tensor([[0., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.]])
labels = torch.tensor([0., 1.])

In [3]:
inputs

tensor([[0.0000, 0.5000, 0.2500, 1.0000],
        [1.0000, 0.5000, 0.2500, 1.0000]])

In [4]:
inputs.shape

torch.Size([2, 4])

In [4]:
labels

tensor([0., 1.])

In [5]:
dataset = TensorDataset(inputs, labels)

In [6]:
dataloader = DataLoader(dataset)

In [7]:
inputs[0].shape

torch.Size([4])

In [8]:
inputs[0].view(len(inputs[0]), 1).shape  # o 1 ao final especifica o numero de colunas como temos 1 linha queremos 1 coluna

torch.Size([4, 1])

In [9]:
# input_size: numero de recursos ou variaveis que temos nos dados de treinamento temos apenas 1
# hidden_size: numero de valores de saída que desejamos, aqui o tamanho do hidden entra como input de uma linear

class LSTMStatQuest(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=1)
        
    def forward(self, x):
        input_trans = x.view(len(x), 1)  # uma linha vira uma coluna
        lstm_out, temp = self.lstm(input_trans)  # lstm_out: contém os valores de curto prazo de cada unidade lstm que desenrolamos isso significa 4 pq precisamos desenrolar o lstm 4x 
# para cada um dos 4 valores de entrada
        prediction = lstm_out[-1]  # por conta disso extraimos o valor da ultima unidade LSTM 
        return prediction

In [10]:
model = LSTMStatQuest()

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
criterion = torch.nn.MSELoss()

In [12]:
len(inputs[0])

4

In [13]:
# Before train:
# exemplo quando 0:
with torch.no_grad():
    pred_0 = model(inputs[0]) 
print(f"Company A: Observed = 0, Predicted = {pred_0}")
# exemplo quando 1:
with torch.no_grad():
    pred_1 = model(inputs[1])
print(f"Company B: Observed = 1, Predicted = {pred_1}")

Company A: Observed = 0, Predicted = tensor([0.1786])
Company B: Observed = 1, Predicted = tensor([0.1740])


In [14]:
EPOCHS = 300
loss_history = []

In [15]:
optimizer.zero_grad(set_to_none=True)
for epoch in range(EPOCHS):
    epoch_loss = 0.0  # Inicializa a loss para a época atual

    for features, labels in dataloader:
        # remove a dimensão adicional
        features = features.squeeze()

        # zera os gradientes:
        optimizer.zero_grad()
        
        # predict do modelo:
        outputs = model(features)
        
        # calcula a loss:
        loss = criterion(outputs, labels)
        
        # backpropagation
        loss.backward()
        
        # updates weights and biases
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    loss_history.append(epoch_loss)
    if (epoch + 1) % 10 == 0: 
        print(f'Época [{epoch + 1}/{EPOCHS}] - Loss: {epoch_loss:.4f}')

Época [10/300] - Loss: 0.2621
Época [20/300] - Loss: 0.2530
Época [30/300] - Loss: 0.2521
Época [40/300] - Loss: 0.2524
Época [50/300] - Loss: 0.2520
Época [60/300] - Loss: 0.2521
Época [70/300] - Loss: 0.2520
Época [80/300] - Loss: 0.2519
Época [90/300] - Loss: 0.2518
Época [100/300] - Loss: 0.2516
Época [110/300] - Loss: 0.2515
Época [120/300] - Loss: 0.2512
Época [130/300] - Loss: 0.2508
Época [140/300] - Loss: 0.2501
Época [150/300] - Loss: 0.2484
Época [160/300] - Loss: 0.2437
Época [170/300] - Loss: 0.2307
Época [180/300] - Loss: 0.2060
Época [190/300] - Loss: 0.1765
Época [200/300] - Loss: 0.1488
Época [210/300] - Loss: 0.1255
Época [220/300] - Loss: 0.1071
Época [230/300] - Loss: 0.0931
Época [240/300] - Loss: 0.0823
Época [250/300] - Loss: 0.0740
Época [260/300] - Loss: 0.0674
Época [270/300] - Loss: 0.0620
Época [280/300] - Loss: 0.0576
Época [290/300] - Loss: 0.0538
Época [300/300] - Loss: 0.0506


In [16]:
# After train:
# exemplo quando 0:
with torch.no_grad():
    pred_0 = model(inputs[0])
pred_0 = torch.round(pred_0).item()
print(f"Company A: Observed = 0, Predicted = {pred_0}")
# exemplo quando 1:
with torch.no_grad():
    pred_1 = model(inputs[1])
pred_1 = torch.round(pred_1).item()
print(f"Company B: Observed = 1, Predicted = {pred_1}")

Company A: Observed = 0, Predicted = 0.0
Company B: Observed = 1, Predicted = 1.0
